In [1]:
# -*- coding: utf-8 -*-

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parse
from time import sleep
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import random
import matplotlib.pyplot as plt
import math
% matplotlib inline

In [3]:
import lxml
import requests
import html5lib
pd.options.display.max_rows = 50

In [116]:
from bokeh.models import HoverTool, CategoricalColorMapper, OpenURL, TapTool
from bokeh.plotting import ColumnDataSource, figure, output_file, show, save, output_notebook
from bokeh.io import curdoc

In [5]:
table_pages = ['https://en.wikipedia.org/wiki/List_of_battles_before_301',
               'https://en.wikipedia.org/wiki/List_of_battles_301%E2%80%931300',
               'https://en.wikipedia.org/wiki/List_of_battles_1301%E2%80%931600',
               'https://en.wikipedia.org/wiki/List_of_battles_since_2001']

In [6]:
list_pages = ['https://en.wikipedia.org/wiki/List_of_battles_1601%E2%80%931800',
               'https://en.wikipedia.org/wiki/List_of_battles_1801%E2%80%931900',
               'https://en.wikipedia.org/wiki/List_of_battles_1901%E2%80%932000',]

In [7]:
def list_battles(url):
    req = requests.get(url)
    print req.status_code
    era = req.content
    era_soup = BeautifulSoup(era)
    bullets = era_soup.find_all('li')
    
    for bullet in bullets:
        print bullet
        if 'title' in str(bullet):
#             if 'expedition' or 'recapture' or 'capture' or 'war' or 'action' or 'battle' or 'siege' or 'operation' or 'fall' in str(bullet).lower():
#                 if 'List+of+battles' not in str(bullet):
                    if 'footer' not in str(bullet):
                        link = bullet.a['href']
                        link_list.append(link)

                        title = bullet.a['title']
                        title_list.append(title)
                    else:
                        print 'NEXT ENTRY: ' + str(bullet)
#                 else:
#                     print 'NEXT ENTRY: ' + str(bullet)
#             else:
#                 print 'NEXT ENTRY: ' + str(bullet)
        else: 
            print 'NEXT ENTRY' + str(bullet)
                    

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(int req.status_code)? (<ipython-input-7-17f2e63ac4c1>, line 3)

In [ ]:
title_list = []
link_list = []
for url in list_pages:
    list_battles(url)

In [ ]:
link_list

In [ ]:
df_late_battles = pd.concat([pd.Series(link_list), pd.Series(title_list)], axis = 1).drop_duplicates()

In [ ]:
df_late_battles = df_late_battles.rename(columns = {0:'url', 1:'Battle'})

In [ ]:
df_late_battles

In [ ]:
df_late_battles[~df_late_battles['Battle'].str.contains('Battle' or 'Operation')]

In [ ]:
df_late_battles.to_csv('battle_list_1600-', encoding = 'utf-8')

In [ ]:
df_late_battles = pd.read_csv('battle_list_1600-.csv', encoding = 'utf-8')

In [ ]:
df_late_battles.head()

In [ ]:
df_late_battles = df_late_battles.drop(['Column1', 'Column4'], axis = 1)
df_late_battles.columns = df_late_battles.iloc[0]
df_late_battles = df_late_battles.drop(0)

In [ ]:
battle_list = []
link_list = []

In [ ]:
def early_battles(link):
    req = requests.get(link)
    print req.status_code
    era = req.content
    era_soup = BeautifulSoup(era)
    tables = era_soup.find_all('table', {'class': 'wikitable'})

    for table in tables:
        cells = table.find_all('td')
        for cell in cells:
            if cell.a is not None:
                a = ['Fall', 'Battle', 'Siege', 'Capture', 'Operation', 'Action', 'Recapture']
                if any(x in str(cell.a['href']) for x in a):
                    battle_list.append(cell.a['title'])
                    link_list.append(cell.a['href'])

In [ ]:
for page in table_pages:
    print page
    early_battles(page)

In [ ]:
df_battles = pd.DataFrame({'battle': battle_list,
                          'link':link_list})

In [ ]:
df_battles = df_battles.rename(columns = {'link':'url', 'battle':'Battle'})

In [ ]:
len(df_battles)

In [ ]:
len(df_late_battles)

In [ ]:
df_battles.head()

In [ ]:
df_url = pd.concat([df_late_battles, df_battles]).reset_index(drop = True)

In [ ]:
len(df_url)

In [ ]:
len(df_url.drop_duplicates())

In [ ]:
s_req = requests.get('https://en.wikipedia.org/Alexander_the_Great')

soup = BeautifulSoup(s_req.text)

ul = soup.find('ul', {'class':'redirectText'})

In [ ]:
if ul is not None:
    print 'works'

In [ ]:
df_battle_all = pd.DataFrame()
df_battle = pd.DataFrame()
def table_scrape(url, name):
#     sleep(1)
    print name
    link = 'https://en.wikipedia.org' + url
    b_req = requests.get(link)

    soup = BeautifulSoup(b_req.text)
    
    bell = soup.find('table', {'class':'infobox vevent'})
    
    if bell is not None:

        details = pd.DataFrame(parse.make2d(bell))
    else:
        return b_req
    

    if 'Belligerents' in str(bell):
    
        clean_rows = details.drop_duplicates(0).set_index(0,drop = True).loc[['Date', 'Location', 'Result'],:]

        clean_col = clean_rows.transpose().reset_index(drop = True)

        df_x_bells = pd.DataFrame()

        html_table = bell.find_all('tr')
        bell_list = []
        bell_y_list = []

        for index, cell in zip(range(0, len(html_table)), html_table):
            if 'Commanders and leaders' in str(cell):
                leaders = html_table[index + 1]
                cells = leaders.find_all('td')
                count = 0
                for cell in cells:
                    count += 1
                    anchors = cell.find_all('a')
                    for anchor in anchors:
                        if 'title' in str(anchor):
                            if 'class="image"' in str(anchor):
                                cl = 'image'
                            elif 'class="thumbborder"' in str(anchor):
                                cl = 'thumb'
                            elif 'cite_note' in str(anchor):
                                cl = 'cite'
                            elif 'disambiguation needed' in str(anchor):
                                cl = 'disam'
                            elif 'cnote_g' in str(anchor):
                                cl = 'cnote'
                            else:
                                title = anchor['title']
                                if any(title == x for x in ['Napoleon I','Alexander III of Macedon']):
                                    print 'napoleon or alex redirect'

                                ref = anchor['href']
                                link = 'https://en.wikipedia.org' + ref

                                if len(ref) <= 10:
                                    t_req = requests.get(link)
                                elif any(ref[8] + ref[9] + ref[10] == x for x in ['pt.', 'tr.', 'ko.', 'ja.', 'th.','da.', 'es.', 'de.', 'it.', 'fr.', 'zh.']):
                                    print 'Not English!'
                                    link = ref
                                    t_req = requests.get(link)
                                elif soup.find('ul', {'class':'redirectText'}) is not None:
                                    print 'REDIRECT'
                                    print title
                                    ul = soup.find('ul', {'class':'redirectText'})
                                    a = ul.a
                                    redirect = a['href']
                                    link = 'https://en.wikipedia.org' + redirect
                                    t_req = requests.get(link)
                                else:
                                    t_req = requests.get(link)

                                t_soup = BeautifulSoup(t_req.text)

                                c_name = t_soup.find('title')
                                c_name = str(c_name)

                                if c_name:
                                    c_name = c_name.replace(' - Wikipedia', '').replace('<title>', '').replace('</title>', '')
                                else:
                                    if count == 1:
                                        bell_list.append(title)
                                    elif count == 2:
                                        bell_y_list.append(title)
                                if 'ex.php?title=' in ref:
                                    ref = ref.replace('ex.php?title=', '')
                                    ref = ref.replace('&action=edit&redlink=1', '')

                                omit_list = ['Capital punishment', 
                                             'Military advisor', 
                                             'Wounded in action', 
                                             'Prisoner of war', 
                                             'Killed in action', 
                                             'Surrender (military)', 
                                             'Surrendered',
                                             'Common military ranks in English']

                                if all(c_name != x for x in omit_list):


                                    if count == 1:

                                        bell_list.append(c_name)
                                    elif count == 2:
                                        bell_y_list.append(c_name)



        for bell in bell_list:
            clean_col['belligerent'] = bell
            df_x_bells = pd.concat([df_x_bells, clean_col])
            df_x_bells = df_x_bells.reset_index(drop = True)


        df_y_bells = pd.DataFrame()
        for bell in bell_y_list:
            clean_col['belligerent'] = bell
            df_y_bells = pd.concat([df_y_bells, clean_col])
            df_y_bells = df_y_bells.reset_index(drop = True)

        df_y_bells = df_y_bells.rename(columns = {0:'Location'})

        if 'Strength' in str(details[0]):
            strength_finder = details[details[0] == 'Strength']


            strength_row = strength_finder.index + 1

            strength = details.loc[strength_row]

            strength_x = strength.iloc[0,0]

            strength_y = strength.iloc[0,1]
        else:
            strength_x = 'unknown'
            strength_y = 'unknown'

        df_x_bells['own'] = strength_x

        df_x_bells['opp'] = strength_y

        df_y_bells['own'] = strength_y

        df_y_bells['opp'] = strength_x

        if 'Casualties and losses' not in str(details[0]):
            df_x_bells['taken'] = np.NaN
            df_x_bells['inflicted'] = np.NaN
            df_y_bells['taken'] = np.NaN
            df_y_bells['inflicted'] = np.NaN

        else:

            casualties_finder = details[details[0] == 'Casualties and losses']

            casualties_row = casualties_finder.index + 1

            casualties = details.loc[casualties_row]

            casualties_x = casualties.iloc[0,0]

            casualties_y = casualties.iloc[0,1]

            df_x_bells['taken'] = casualties_x

            df_x_bells['inflicted'] = casualties_y

            df_y_bells['taken'] = casualties_y

            df_y_bells['inflicted'] = casualties_x



        df_x_bells['Battle'] = name

        df_y_bells['Battle'] = name

        df_x_bells['pos'] = 'L'

        df_y_bells['pos'] = 'R'

        df_battle = pd.concat([df_x_bells, df_y_bells]).reset_index(drop = True)

        df_battle_all = pd.concat([df_battle_all, df_battle]).reset_index(drop = True).drop_duplicates()

        global df_battle_all
    else:
        print 'Filtered: ' + name

In [ ]:
count = 0
for num, rows in {1:df_url[:200],
             2:df_url[200:400], 
             3:df_url[400:600], 
             4:df_url[600:800],
             5:df_url[800:1000],
             6:df_url[1000:1200],
             7:df_url[1200:1400],
             8:df_url[1400:1600],
             9:df_url[1600:1800],
             10:df_url[1800:2000],
             11:df_url[2000:2200],
             12:df_url[2200:2400],
             13:df_url[2400:2600],
             14:df_url[2600:2800],
             15:df_url[2800:3000],
             16:df_url[3000:3200],
             17:df_url[3200:3400],
             18:df_url[3400:]}.iteritems():
    df_battle = pd.DataFrame()
    df_batch = rows
    for index, row in df_batch.iterrows():
        count += 1
        global count
        print str(count) + '/' + str(len(df_url))
        table_scrape(row['url'], row['Battle'])
    df_battle_all.to_csv('rows_{}.csv'.format(num), encoding = 'utf-8')
    print 'NEW FRAME' + str(num)

In [ ]:
row_list = ['rows_1.csv',
 'rows_2.csv',
 'rows_3.csv',
 'rows_4.csv',
 'rows_5.csv',
 'rows_6.csv',
 'rows_7.csv',
 'rows_8.csv',
 'rows_9.csv',
 'rows_10.csv',
 'rows_11.csv',
 'rows_12.csv',
 'rows_13.csv',
 'rows_14.csv',
 'rows_15.csv',
 'rows_16.csv',
 'rows_17.csv',
 'rows_18.csv']

In [ ]:
df_comm = pd.DataFrame()
for sheet in row_list:
    df_rows = pd.read_csv(sheet)
    df_comm = pd.concat([df_comm, df_rows]).reset_index(drop = True)

In [ ]:
df_comm = df_comm.drop('Unnamed: 0', axis = 1)

In [ ]:
len(df_comm)

In [ ]:
len(df_comm)

In [ ]:
df_comm.to_csv('battles_dirty.csv', encoding = 'utf-8')

In [ ]:
df_comm = pd.read_csv('battles_dirty.csv').drop('Unnamed: 0', axis = 1)

In [ ]:
df_comm = df_comm.drop_duplicates()

In [ ]:
df_comm.head()

In [ ]:
len(df_comm)

In [ ]:
df_comm[df_comm.belligerent.str.contains('Nasser')]

In [ ]:
len(df_comm.drop_duplicates())

In [ ]:
df_comm = df_comm.drop_duplicates().reset_index(drop = True)

In [ ]:
df_comm.to_csv('battles_deduped.csv')

In [ ]:
len(df_comm[df_comm.belligerent.str.contains('Napoleon')])

In [ ]:
df_comm[df_comm.belligerent.str.contains('Scipio')]

In [ ]:
df_results = pd.read_csv('result_column.csv')

In [ ]:
df_results[df_results.Battle == 'Battle_of_Camarón']

In [ ]:
df_comm.head()

In [ ]:
df_results.head()

In [ ]:
df_more_results = pd.merge(df_comm, df_results, how = 'outer', on = ['Battle', 'belligerent', 'pos', 'Result'])

In [ ]:
df_more_results = df_more_results[df_more_results.VorD.isnull()]

In [ ]:
df_more_results = df_more_results.drop_duplicates(subset = ['Battle', 'Date', 'VorD'])

In [ ]:
df_more_results.head()

In [ ]:
df_more_results.to_csv('adding_results_10.21.csv', encoding = 'utf-8')

In [ ]:
df_comm[df_comm.Battle == 'Battle_of_Camarón']

In [ ]:
df_comm[df_comm.belligerent == 'Ambroise de Loré']

In [ ]:
df_vl = pd.merge(df_comm, df_results, how = 'outer', on = ['Battle', 'belligerent', 'Result', 'pos'])

In [ ]:
df_vl

In [ ]:
df_vl = df_vl.sort('VorD')

In [ ]:
df_vl[df_vl.belligerent == 'Ambroise de Loré']

In [ ]:
df_vl[df_vl.belligerent == 'Saracens']

In [ ]:
df_vl.to_csv('additional_results.csv')

In [ ]:
df_10_21_fill = df_vl[df_vl.VorD.isnull()]

In [ ]:
df_ar = pd.read_csv('additional_results.csv', encoding = 'utf-8')

In [ ]:
df_strength = df_ar.drop(['Unnamed: 0', 'Location', 'Result', 'inflicted', 'taken', 'opp'], axis = 1)

In [ ]:
df_strength = df_strength.sort('Battle').reset_index(drop = True)

In [ ]:
df_strength_dd = df_strength.drop_duplicates(['own', 'pos'], keep = 'first')

In [ ]:
df_strength_dd.own = df_strength_dd.own.str.replace('\r',' ').str.replace('\n', ' ').replace('\n', ' ')

In [ ]:
df_strength_dd = df_strength_dd.sort('Battle').reset_index(drop = True)

In [ ]:
df_strength_dd

In [ ]:
df_strength_dd.to_csv('strength_entry.csv', encoding = 'utf-8')

In [ ]:
df_strength_new = df_strength.drop_duplicates(['own', 'pos', 'Battle'], keep = 'first')

In [ ]:
df_strength_fill = pd.read_excel('strength_entry.xlsx', encoding = 'utf-8')

In [ ]:
df_battles = pd.read_csv('battles_deduped.csv', encoding = 'utf-8').drop('Unnamed: 0', axis = 1)

In [ ]:
df_battles = df_battles.drop(['taken', 'inflicted'], axis = 1)

In [ ]:
df_strength_fill.Battle = df_strength_fill.Battle.str.replace(u"â€“",u'–')

In [ ]:
df_some_strength = pd.merge(df_battles, df_strength_fill, how = 'outer', on = ['Battle', 'pos']).drop(['own_y', 'belligerent_y', 'Date_y'], axis = 1).rename(columns = {'Date_x':'Date','own_x':'own', 'belligerent_x':'belligerent'})

In [ ]:
df_some_strength[df_some_strength.Battle.str.contains('Battle_of_Jena')]

In [ ]:
df_test = df_some_strength.iloc[:,9:]

In [ ]:
df_str_two = df_some_strength[df_some_strength['Infantry'].isnull() & df_some_strength['Cavalry'].isnull() & df_some_strength['Artillery'].isnull() & df_some_strength['Ships'].isnull() & df_some_strength['Airforce'].isnull() & df_some_strength['Special'].isnull()]

In [ ]:
df_str_two.head()

In [ ]:
df_str_two = df_str_two.drop_duplicates(['Battle', 'Date', 'Location', 'Result', 'pos']).drop('VorD', axis = 1)

In [ ]:
len(df_str_two)

In [ ]:
df_merger = df_strength.drop(['Date', 'belligerent', 'own'], axis = 1)

In [ ]:
df_merger = df_merger.drop_duplicates()

In [ ]:
df_str_two.head()

In [ ]:
df_str_two = pd.merge(df_str_two, df_merger, on = ['Battle', 'pos'], how = 'outer').dropna(subset = ['Date', 'Location', 'Result'])

In [ ]:
df_str_two = df_str_two.dropna(subset = ['VorD'])

In [ ]:
df_str_two.to_csv('strength_entry_two.csv')

In [ ]:
df_str_two

In [ ]:
df_partial = pd.read_excel('PARTIAL_strength_entry_two.xlsx', encoding = 'utf-8')

In [ ]:
df_str_two.head()

In [ ]:
df_str_two = df_str_two.drop(['Infantry', 'Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special'], axis = 1)

In [ ]:
df_partial = df_partial.drop(['Date', 'Location', 'Result', 'belligerent','own', 'Column1'],axis = 1)

In [ ]:
df_partial.columns

In [ ]:
df_str_two.head()

In [ ]:
df_finish = pd.merge(df_str_two, df_partial, on = ['Battle', 'pos', 'VorD'], how = 'outer')

In [ ]:
drop_list = list(df_finish[df_finish['Date'].isnull()].index)

In [ ]:
df_finish = df_finish.drop(drop_list)

In [ ]:
df_finish.to_csv('last_strength_entry.csv', encoding = 'utf-8')

In [ ]:
df_some_strength

In [ ]:
df_some_strength[df_some_strength.Battle.str.contains('Siege of La Rochelle')]

In [ ]:
df_more_strength = pd.read_excel('last_strength_entry.xlsx')

In [ ]:
df_more_strength.head()

In [ ]:
df_some_strength.head()

In [ ]:
df_more_strength = pd.merge(df_more_strength, df_str_two, on = ['Battle', 'pos'], how = 'outer').drop(['own_y', 'Infantry_y', 'Cavalry_y', 'Artillery_y', 'Ships_y', 'Airforce_y', 'Special_y', 'Date_y', 'Location_y', 'Result_y', 'belligerent_y'], axis = 1).dropna(subset = ['Date_x'])

In [ ]:
df_more_strength.columns = df_more_strength.columns.str.replace('_x', '')

In [ ]:
df_more_strength.head()

In [ ]:
df_some_strength.head()

In [ ]:
df_more_strength

In [ ]:
df_fill_two = pd.merge(df_battles, df_more_strength, on = ['Battle', 'pos'], how = 'outer').drop(['Date_y', 
                                                                                    'Location_y', 
                                                                                    'belligerent_y', 
                                                                                    'Result_y', 'own_y', 
                                                                                    'opp_y'], axis = 1)

In [ ]:
df_fill_two.columns = df_fill_two.columns.str.replace('_x', '')

In [ ]:
df_some_strength = df_some_strength[df_some_strength['Infantry'].notnull() | df_some_strength['Cavalry'].notnull() | df_some_strength['Artillery'].notnull() | df_some_strength['Ships'].notnull() | df_some_strength['Airforce'].notnull() | df_some_strength['Special'].notnull()]

In [ ]:
len(df_some_strength)

In [ ]:
df_fill_two = df_fill_two[df_fill_two['Infantry'].notnull() | df_fill_two['Cavalry'].notnull() | df_fill_two['Artillery'].notnull() | df_fill_two['Ships'].notnull() | df_fill_two['Airforce'].notnull() | df_fill_two['Special'].notnull()]

In [ ]:
df_some_strength

In [ ]:
df_fill_two

In [ ]:
df_all_strength = pd.concat([df_fill_two, df_some_strength]).reset_index(drop = True)

In [ ]:
len(df_battles)

In [ ]:
df_char = df_all_strength[df_all_strength.Date.isnull()]

In [ ]:
df_char.to_excel('special_character_fix.xlsx')

In [ ]:
df_all_strength[df_all_strength.Battle.str.contains('Battle_of_Tenn')]

In [ ]:
df_all_strength.to_excel('all_strength_probably.xlsx')

In [ ]:
df_battles[df_battles.Battle.str.contains('dlingen')]

In [ ]:
df_char.Battle

In [8]:
df_strength_all = pd.read_excel('all_strength_probably.xlsx')

In [9]:
df_strength_all = df_strength_all.drop_duplicates()

In [10]:
df_left = df_strength_all[df_strength_all.pos == 'L']

In [11]:
df_left = df_left.drop(['Date', 'Location', 'Result', 'belligerent', 'opp', 'own'], axis = 1)

In [12]:
df_left = df_left.drop_duplicates(subset = ['Battle'])

In [13]:
df_left = df_left.dropna(subset = ['Infantry', 'Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special'], how = 'all')

In [14]:
df_right = df_strength_all[df_strength_all.pos == 'R']

In [15]:
df_right = df_right.drop_duplicates(subset = ['Battle'])

In [16]:
df_right = df_right.drop(['Date', 'Location', 'Result', 'belligerent', 'opp', 'own', 'VorD'], axis = 1)

In [17]:
df_right = df_right.dropna(subset = ['Infantry', 'Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special'], how = 'all')

In [18]:
df_model = pd.merge(df_left, df_right, on = 'Battle', how = 'outer').dropna(subset = ['pos_x', 'pos_y']).reset_index(drop = True)

In [19]:
df_model

,Battle,pos_x,VorD,Infantry_x,Cavalry_x,Artillery_x,Ships_x,Airforce_x,Special_x,pos_y,Infantry_y,Cavalry_y,Artillery_y,Ships_y,Airforce_y,Special_y
0,1898_invasion_of_GuantÃ¡namo_Bay,L,V,923.0,NaN,NaN,6.0,NaN,NaN,R,5000.0,NaN,3.0,2.0,NaN,NaN
1,1948_Arab–Israeli_War,L,V,117500.0,NaN,NaN,NaN,NaN,NaN,R,58800.0,NaN,NaN,NaN,NaN,NaN
2,1982_Lebanon_War,L,I,113000.0,2397.0,NaN,NaN,634.0,NaN,R,37500.0,882.0,1125.0,NaN,450.0,NaN
3,1998_Battle_of_Kilinochchi,L,D,1.0,NaN,NaN,NaN,NaN,NaN,R,1.0,NaN,NaN,NaN,NaN,NaN
4,Abagana_Ambush,L,D,6000.0,NaN,NaN,NaN,NaN,NaN,R,700.0,NaN,NaN,NaN,NaN,NaN
5,Action_at_Bronkhorstspruit,L,D,268.0,NaN,NaN,NaN,NaN,NaN,R,250.0,NaN,NaN,NaN,NaN,NaN
6,Attack_on_Rabaul,L,V,NaN,NaN,NaN,16.0,631.0,NaN,R,NaN,NaN,NaN,21.0,200.0,NaN
7,Attack_on_Sydney_Harbour,L,I,NaN,NaN,NaN,19.0,NaN,NaN,R,NaN,NaN,NaN,8.0,2.0,NaN
8,"Battle of ""The Saw""",L,V,10000.0,2000.0,NaN,NaN,NaN,NaN,R,50000.0,NaN,NaN,NaN,NaN,NaN
9,Battle of Ã‡Ä±ldÄ±r,L,V,1.0,NaN,NaN,NaN,NaN,NaN,R,32000.0,NaN,NaN,NaN,NaN,NaN


In [20]:
df_model.VorD.value_counts()

V    1524
D     775
I     173
v       1
L       1
Name: VorD, dtype: int64

In [21]:
(1524 - 775) / 2

374.5

In [22]:
df_v = df_model[df_model.VorD == 'V']

In [23]:
v_list = list(df_v.index)

In [24]:
ran_list = random.sample(v_list, 374)

In [25]:
df_switch = df_model.iloc[ran_list]

In [26]:
df_switch['Infantry_hold'] = df_switch['Infantry_x']
df_switch['Cavalry_hold'] = df_switch['Cavalry_x']
df_switch['Artillery_hold'] = df_switch['Artillery_x']
df_switch['Ships_hold'] = df_switch['Ships_x']
df_switch['Airforce_hold'] = df_switch['Airforce_x']
df_switch['Special_hold'] = df_switch['Special_x']

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [27]:
df_switch['Infantry_x'] = df_switch['Infantry_y']
df_switch['Cavalry_x'] = df_switch['Cavalry_y']
df_switch['Artillery_x'] = df_switch['Artillery_y']
df_switch['Ships_x'] = df_switch['Ships_y']
df_switch['Airforce_x'] = df_switch['Airforce_y']
df_switch['Special_x'] = df_switch['Special_y']

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [28]:
df_switch['Infantry_y'] = df_switch['Infantry_hold']
df_switch['Cavalry_y'] = df_switch['Cavalry_hold']
df_switch['Artillery_y'] = df_switch['Artillery_hold']
df_switch['Ships_y'] = df_switch['Ships_hold']
df_switch['Airforce_y'] = df_switch['Airforce_hold']
df_switch['Special_y'] = df_switch['Special_hold']

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [29]:
df_switch = df_switch.drop(['Infantry_hold', 'Cavalry_hold', 'Artillery_hold', 'Ships_hold', 'Airforce_hold', 'Special_hold'], axis = 1)

In [30]:
df_switch.VorD = 'D'

In [31]:
df_switch.VorD.value_counts()

D    374
Name: VorD, dtype: int64

In [32]:
df_model = df_model.drop(ran_list)

In [33]:
df_model = pd.concat([df_model, df_switch])

In [34]:
df_model.VorD.value_counts()

V    1150
D    1149
I     173
v       1
L       1
Name: VorD, dtype: int64

In [35]:
df_model.loc[2378, 'Infantry_y'] = 25000

In [36]:
df_model.loc[2363, 'Infantry_x'] = 500

In [37]:
df_model = df_model[df_model.Infantry_x != 1.0]
df_model = df_model[df_model.Infantry_y != 1.0]

In [38]:
df_model = df_model.fillna(0)

In [39]:
df_model['infantry_diff'] = (df_model['Infantry_x'] - df_model['Infantry_y']) / (df_model['Infantry_x'] + df_model['Infantry_y'])
df_model['cavalry_diff'] = (df_model['Cavalry_x'] - df_model['Cavalry_y']) / (df_model['Cavalry_x'] + df_model['Cavalry_y'])
df_model['artillery_diff'] = (df_model['Artillery_x'] - df_model['Artillery_y']) / (df_model['Artillery_x'] + df_model['Artillery_y'])
df_model['ships_diff'] = (df_model['Ships_x'] - df_model['Ships_y']) / (df_model['Ships_x'] + df_model['Ships_y'])
df_model['airforce_diff'] = (df_model['Airforce_x'] - df_model['Airforce_y']) / (df_model['Airforce_x'] + df_model['Airforce_y'])
df_model['special_diff'] = (df_model['Special_x'] - df_model['Special_y']) / (df_model['Special_x'] + df_model['Special_y'])

In [40]:
df_diff = pd.concat([df_model['VorD'], df_model['Battle'], df_model['infantry_diff'],df_model['cavalry_diff'],df_model['artillery_diff'], df_model['ships_diff'],
                    df_model['airforce_diff'], df_model['special_diff']], axis = 1)

In [41]:
df_diff = df_diff[df_diff.VorD != 'I']

In [42]:
df_model[df_model.Battle == 'Siege of Noda']

,Battle,pos_x,VorD,Infantry_x,Cavalry_x,Artillery_x,Ships_x,Airforce_x,Special_x,pos_y,...,Artillery_y,Ships_y,Airforce_y,Special_y,infantry_diff,cavalry_diff,artillery_diff,ships_diff,airforce_diff,special_diff
2363,Siege of Noda,L,V,500.0,0.0,0.0,0.0,0.0,0.0,R,...,0.0,0.0,0.0,0.0,0.818182,NaN,NaN,NaN,NaN,NaN


In [43]:
df_diff.sort_values('infantry_diff', ascending = False)

,VorD,Battle,infantry_diff,cavalry_diff,artillery_diff,ships_diff,airforce_diff,special_diff
544,V,Battle of Pontvallain,1.000000,-1.000000,NaN,NaN,NaN,NaN
164,D,Battle of Carrhae,1.000000,-0.428571,NaN,NaN,NaN,NaN
644,V,Battle of the Arius,1.000000,-1.000000,NaN,NaN,NaN,NaN
476,V,Battle of NÄ›meckÃ½ Brod,1.000000,-1.000000,NaN,NaN,NaN,NaN
506,D,Battle of Obertyn,1.000000,-0.582573,-0.612903,NaN,NaN,NaN
327,D,Battle of Ikh Bayan,1.000000,-1.000000,NaN,NaN,NaN,NaN
578,D,Battle of Salsu,1.000000,-1.000000,NaN,NaN,NaN,NaN
1136,D,Battle_of_Elizabeth_City,1.000000,NaN,1.000000,-0.368421,NaN,NaN
152,D,Battle of Callinicum,1.000000,-0.600000,NaN,NaN,NaN,NaN
431,V,Battle of Marj Rahit (684),1.000000,-1.000000,NaN,NaN,NaN,NaN


In [44]:
df_diff = df_diff.drop(174)

In [45]:
df_diff = df_diff.fillna(0)

In [46]:
df_x = pd.DataFrame(df_diff.iloc[:, 2:])

In [47]:
df_x = df_x.drop('artillery_diff', axis = 1)

In [48]:
df_y = pd.DataFrame(df_diff.iloc[:, 0])

In [49]:
df_y.VorD = df_y.VorD.replace('V', 1).replace('D', 0).replace('L', 0).replace('v', 1)

In [50]:
df_y.VorD = df_y.VorD.astype('float')

In [51]:
df_x = df_x.iloc[:, :-1]

In [52]:
lr = LogisticRegression()
lr.fit(df_x, df_y)

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [53]:
lr.score(df_x, df_y)

0.54048582995951422

In [54]:
pd.Series(df_x.columns)

0    infantry_diff
1     cavalry_diff
2       ships_diff
3    airforce_diff
dtype: object

In [55]:
lr.coef_[0]

array([ 0.40991453,  0.09871432,  0.57318956,  0.11971539])

In [56]:
pd.concat([pd.Series(lr.coef_[0]), pd.Series(df_x.columns)], axis = 1)

,0,1
0,0.409915,infantry_diff
1,0.098714,cavalry_diff
2,0.573190,ships_diff
3,0.119715,airforce_diff


In [57]:
df_test = pd.concat([df_y.reset_index(drop = True), pd.Series(lr.predict(df_x))], axis = 1)

In [58]:
df_test.head()

,VorD,0
0,1.0,1.0
1,0.0,1.0
2,0.0,1.0
3,1.0,0.0
4,1.0,0.0


In [59]:
len(df_test)

1976

In [60]:
count = 0
for i, row in df_test.iterrows():
    if row['VorD'] == row[0]:
        count = count + 1

In [61]:
scores = cross_val_score(lr, df_x, df_y.VorD, cv = 10)

In [62]:
scores.mean()

0.53699687227606008

In [63]:
pd.concat([pd.DataFrame(lr.predict_proba(df_x)), df_x.reset_index(drop = True)] ,axis = 1)

,0,1,infantry_diff,cavalry_diff,ships_diff,airforce_diff
0,0.464437,0.535563,0.332955,0.000000,0.000000,0.000000
1,0.418174,0.581826,0.791045,0.000000,0.000000,0.000000
2,0.494936,0.505064,0.034749,0.000000,0.000000,0.000000
3,0.542038,0.457962,-0.666667,1.000000,0.000000,0.000000
4,0.500177,0.499823,-0.016393,0.000000,0.000000,0.000000
5,0.516604,0.483396,-0.073171,0.000000,-0.074074,0.000000
6,0.482738,0.517262,0.153846,0.000000,0.000000,0.000000
7,0.498497,0.501503,0.000000,0.000000,0.000000,0.000000
8,0.538789,0.461211,-0.393939,0.000000,0.000000,0.000000
9,0.447453,0.552547,0.500000,0.000000,0.000000,0.000000


In [64]:
lr.coef_

array([[ 0.40991453,  0.09871432,  0.57318956,  0.11971539]])

In [65]:
df_test = pd.DataFrame({'infantry_diff':.99,
                       'cavalry_diff':.99,
                       'artillery_diff':.99,
                       'ships_diff':.99,
                       'airforce_diff:':.99}, index = [0])

In [66]:
df_test

,airforce_diff:,artillery_diff,cavalry_diff,infantry_diff,ships_diff
0,0.99,0.99,0.99,0.99,0.99


In [67]:
lr.predict_proba(df_test)

ValueError: X has 5 features per sample; expecting 4

In [ ]:
df_model.VorD.value_counts()

In [68]:
1524 + 775

2299

In [69]:
2300 /2 - 775

375.0

In [70]:
rf = RandomForestClassifier()
rf.fit(df_x, df_y)

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [71]:
rf.score(df_x, df_y)

0.83198380566801622

In [72]:
scores = cross_val_score(rf, df_x, df_y.VorD, cv = 5)

In [73]:
scores

array([ 0.52272727,  0.52405063,  0.56455696,  0.49367089,  0.4835443 ])

In [74]:
rf.feature_importances_

array([ 0.75045784,  0.11047607,  0.11331901,  0.02574708])

In [75]:
pd.concat([pd.DataFrame(rf.predict_proba(df_x)), df_x.reset_index(drop = True)], axis = 1)

,0,1,infantry_diff,cavalry_diff,ships_diff,airforce_diff
0,0.168483,0.831517,0.332955,0.000000,0.000000,0.000000
1,1.000000,0.000000,0.791045,0.000000,0.000000,0.000000
2,0.900000,0.100000,0.034749,0.000000,0.000000,0.000000
3,0.383333,0.616667,-0.666667,1.000000,0.000000,0.000000
4,0.000000,1.000000,-0.016393,0.000000,0.000000,0.000000
5,0.000000,1.000000,-0.073171,0.000000,-0.074074,0.000000
6,0.325000,0.675000,0.153846,0.000000,0.000000,0.000000
7,0.611440,0.388560,0.000000,0.000000,0.000000,0.000000
8,1.000000,0.000000,-0.393939,0.000000,0.000000,0.000000
9,0.580516,0.419484,0.500000,0.000000,0.000000,0.000000


In [76]:
df_strength_all.loc[60, 'VorD'] = 'V'
df_strength_all.loc[61, 'VorD'] = 'V'

In [77]:
df_strength_all.loc[3479:3485, 'VorD'] = 'V'

In [78]:
df_strength_all.loc[3485, 'VorD'] = 'D'

In [79]:
df_strength_all.loc[3802:3803, 'VorD'] = 'V'
df_strength_all.loc[3804:3807, 'VorD'] = 'D'

In [80]:
df_strength_all.loc[4142:4143, 'VorD'] = 'D'

In [81]:
df_strength_all.VorD = df_strength_all.VorD.str.replace('L', 'D')

In [82]:
df_strength_all.VorD = df_strength_all.VorD.str.replace('v', 'V')

In [83]:
df_strength_all.VorD = df_strength_all.VorD.str.replace('W', 'V')

In [84]:
null_fill = df_strength_all[df_strength_all.VorD.isnull()]

In [85]:
null_fill

,Battle,Date,Location,Result,belligerent,opp,own,pos,VorD,Infantry,Cavalry,Artillery,Ships,Airforce,Special
4981,Battle_of_Gadebusch,9 December 1712 (O.S.)\r\r\r\n20 December 1712...,"Wakenstädt near Gadebusch, 35 km southern of L...",Swedish victory,Sweden,"17,000:[b]\r\r\r\n8,300 Danish foot,\r\r\r\n4,...","12,500:[a]\r\r\r\n6,500 foot,\r\r\r\n6,000 hor...",L,NaN,6500.0,6000.0,25.0,NaN,NaN,NaN
4982,Battle_of_Gadebusch,9 December 1712 (O.S.)\r\r\r\n20 December 1712...,"Wakenstädt near Gadebusch, 35 km southern of L...",Swedish victory,Magnus Stenbock,"17,000:[b]\r\r\r\n8,300 Danish foot,\r\r\r\n4,...","12,500:[a]\r\r\r\n6,500 foot,\r\r\r\n6,000 hor...",L,NaN,6500.0,6000.0,25.0,NaN,NaN,NaN
5100,Battle_of_Goose_Green,28–29 May 1982,"Goose Green and Darwin, Falkland Islands",British victory,Ítalo Piaggi,690[6],684[4]–871 army\r\r\r\n202 airforce\r\r\r\n10 ...,L,NaN,990.0,NaN,NaN,NaN,NaN,NaN
5101,Battle_of_Goose_Green,28–29 May 1982,"Goose Green and Darwin, Falkland Islands",British victory,Wing commander (rank),690[6],684[4]–871 army\r\r\r\n202 airforce\r\r\r\n10 ...,L,NaN,990.0,NaN,NaN,NaN,NaN,NaN
6236,Battle_of_Meissen,4 December 1759,"Meissen, Saxony, present-day Germany",Austrian victory,Habsburg Monarchy,"26,000","21,000",L,NaN,21000.0,NaN,NaN,NaN,NaN,NaN
6237,Battle_of_Meissen,4 December 1759,"Meissen, Saxony, present-day Germany",Austrian victory,Count Leopold Joseph von Daun,"26,000","21,000",L,NaN,21000.0,NaN,NaN,NaN,NaN,NaN
6307,Battle_of_Mobile_Bay,"August 2, 1864 (1864-08-02)–August 23, 1864 (1...","Mobile Bay, Alabama",Union victory,David Farragut,"3 gunboats\r\r\r\n1 ironclad\r\r\r\n1,500 men",12 wooden ships\r\r\r\n2 gunboats\r\r\r\n4 iro...,L,NaN,5500.0,NaN,NaN,18.0,NaN,NaN
6308,Battle_of_Mobile_Bay,"August 2, 1864 (1864-08-02)–August 23, 1864 (1...","Mobile Bay, Alabama",Union victory,Union Navy,"3 gunboats\r\r\r\n1 ironclad\r\r\r\n1,500 men",12 wooden ships\r\r\r\n2 gunboats\r\r\r\n4 iro...,L,NaN,5500.0,NaN,NaN,18.0,NaN,NaN
6309,Battle_of_Mobile_Bay,"August 2, 1864 (1864-08-02)–August 23, 1864 (1...","Mobile Bay, Alabama",Union victory,Gordon Granger,"3 gunboats\r\r\r\n1 ironclad\r\r\r\n1,500 men",12 wooden ships\r\r\r\n2 gunboats\r\r\r\n4 iro...,L,NaN,5500.0,NaN,NaN,18.0,NaN,NaN
6310,Battle_of_Mobile_Bay,"August 2, 1864 (1864-08-02)–August 23, 1864 (1...","Mobile Bay, Alabama",Union victory,Union Army,"3 gunboats\r\r\r\n1 ironclad\r\r\r\n1,500 men",12 wooden ships\r\r\r\n2 gunboats\r\r\r\n4 iro...,L,NaN,5500.0,NaN,NaN,18.0,NaN,NaN


In [86]:
null_fill['VorD'] = ['V', 'D', 'D', 'V', 'V', 'D', 'V', 'V', 'V', 'V', 'D', 'D', 'D', 'D', 'V', 'V', 'V', 'V', 'V',
                    'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D','D', 'D', 'D', 'D', 'V', 'V', 'V', 'V', 'V', 'D', 'D', 'D', 'D',
                    'V', 'V', 'D', 'D', 'D', 'D', np.NaN, 'D', np.NaN]

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [87]:
df_strength_all.VorD.loc[null_fill.index] = null_fill['VorD']

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [88]:
df_strength_all = df_strength_all[df_strength_all.belligerent != 'Sweden']

In [89]:
import re

In [90]:
year_list = []
for i, row in df_strength_all.iterrows():
    d = row['Date']
    if isinstance(d, str):
        if re.search(r'[12]\d{3}', d):
            year = re.search(r'[12]\d{3}', d).group(0)
            year = int(year)
            if 'BC' in d:
                year = year * -1
            year_list.append({'Year':year, 'Index':i})
        elif re.search(r'[12]\d{2}', d):
            year = re.search(r'[12]\d{2}', d).group(0)
            year = int(year)
            if 'BC' in d:
                year = year * -1
            year_list.append({'Year':year, 'Index':i})
    

In [91]:
df_years = pd.DataFrame(year_list).set_index('Index', drop = True)

In [92]:
df_strength_all['year'] = pd.Series()

In [93]:
df_strength_all['year'].loc[df_years.index] = df_years.Year

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [94]:
df_strength_all.to_csv('current_run.csv', encoding = 'utf-8')

In [95]:
df_strength_all = pd.read_csv('current_run.csv', encoding = 'utf-8').drop('Unnamed: 0', axis = 1)

In [96]:
df_run = df_strength_all[df_strength_all.VorD.notnull()]

In [97]:
df_run.Battle = df_run.Battle.str.replace('_'," ")

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [98]:
df_run

,Battle,Date,Location,Result,belligerent,opp,own,pos,VorD,Infantry,Cavalry,Artillery,Ships,Airforce,Special,year
0,1898 invasion of GuantÃ¡namo Bay,NaN,NaN,NaN,NaN,NaN,NaN,R,D,5000.0,NaN,3.0,2.0,NaN,NaN,NaN
1,1898 invasion of GuantÃ¡namo Bay,NaN,NaN,NaN,NaN,NaN,NaN,R,D,5000.0,NaN,7.0,2.0,NaN,NaN,NaN
2,1898 invasion of GuantÃ¡namo Bay,NaN,NaN,NaN,NaN,NaN,NaN,L,V,923.0,NaN,NaN,6.0,NaN,NaN,NaN
3,1948 Arab–Israeli War,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...","Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,David Ben-Gurion,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,V,117500.0,NaN,NaN,NaN,NaN,NaN,1948.0
4,1948 Arab–Israeli War,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...","Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,Yisrael Galili,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,V,117500.0,NaN,NaN,NaN,NaN,NaN,1948.0
5,1948 Arab–Israeli War,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...","Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,Yaakov Dori,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,V,117500.0,NaN,NaN,NaN,NaN,NaN,1948.0
6,1948 Arab–Israeli War,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...","Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,Yigael Yadin,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,V,117500.0,NaN,NaN,NaN,NaN,NaN,1948.0
7,1948 Arab–Israeli War,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...","Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,Mickey Marcus,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,V,117500.0,NaN,NaN,NaN,NaN,NaN,1948.0
8,1948 Arab–Israeli War,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...","Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,Yigal Allon,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,V,117500.0,NaN,NaN,NaN,NaN,NaN,1948.0
9,1948 Arab–Israeli War,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...","Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,Yitzhak Rabin,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,V,117500.0,NaN,NaN,NaN,NaN,NaN,1948.0


In [99]:
arras_1 = {'Battle':['Battle of Arras', 'Battle of Arras'], 
           'Date':['21 May 1940', '21 May 1940'],
           'Location':['Arras', 'Arras'], 
           'Result':['German victory', 'German victory'] ,
           'belligerent':['Erwin Rommel', 'Harold Franklyn'],
           'opp':['manual', 'manual'],
           'own':['manual', 'manual'], 
           'pos':['R', 'L'],
           'VorD':['V', 'D'],
           'Infantry':[7500.0, 2000.0],
           'Cavalry':[225.0, 74.0],
           'Artillery':[np.NaN, np.NaN],
           'Ships': [np.NaN, np.NaN],
           'Airforce': [np.NaN, np.NaN],
           'Special': [np.NaN, np.NaN],
           'year':[1940, 1940]}

In [100]:
df_run = pd.concat([df_run,pd.DataFrame(arras_1, index = [10515,10516])])

In [101]:
df_run

,Airforce,Artillery,Battle,Cavalry,Date,Infantry,Location,Result,Ships,Special,VorD,belligerent,opp,own,pos,year
0,NaN,3.0,1898 invasion of GuantÃ¡namo Bay,NaN,NaN,5000.0,NaN,NaN,2.0,NaN,D,NaN,NaN,NaN,R,NaN
1,NaN,7.0,1898 invasion of GuantÃ¡namo Bay,NaN,NaN,5000.0,NaN,NaN,2.0,NaN,D,NaN,NaN,NaN,R,NaN
2,NaN,NaN,1898 invasion of GuantÃ¡namo Bay,NaN,NaN,923.0,NaN,NaN,6.0,NaN,V,NaN,NaN,NaN,L,NaN
3,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,David Ben-Gurion,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
4,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Yisrael Galili,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
5,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Yaakov Dori,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
6,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Yigael Yadin,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
7,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Mickey Marcus,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
8,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Yigal Allon,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0
9,NaN,NaN,1948 Arab–Israeli War,NaN,"15 May 1948 – 10 March 1949\r\r\r\n(9 months, ...",117500.0,"Former British Mandate of Palestine, Sinai Pen...",Israeli victory\r\r\r\nJordanian partial victo...,NaN,NaN,V,Yitzhak Rabin,"Egypt: 10,000 initially, rising to 20,000[cita...","Israel: 29,677 (initially)\r\r\r\n117,500 (fin...",L,1948.0


In [102]:
len(arras_1)

16

In [103]:
df_lookup = df_strength_all.drop_duplicates(subset = ['Battle', 'pos'])

In [104]:
df_lookup.Battle = df_lookup.Battle.str.replace('_', ' ')

/home/ethanarsht/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [105]:
df_lookup = pd.concat([df_lookup, pd.DataFrame(arras_1, index = [10515,10516])])

In [106]:
df_lookup[df_lookup['Battle'] == 'Six Day War']

,Airforce,Artillery,Battle,Cavalry,Date,Infantry,Location,Result,Ships,Special,VorD,belligerent,opp,own,pos,year
10330,957.0,NaN,Six Day War,2504.0,"June 5–10, 1967 (6 days)",547000.0,Middle East,Decisive Israeli victory,NaN,NaN,D,Gamal Abdel Nasser,"50,000 troops\r\r\r\n214,000 reserves\r\r\r\n3...","Egypt: 240,000\r\r\r\nSyria, Jordan, and Iraq:...",R,1967.0
10342,300.0,NaN,Six Day War,800.0,"June 5–10, 1967 (6 days)",264000.0,Middle East,Decisive Israeli victory,NaN,NaN,V,Levi Eshkol,"Egypt: 240,000\r\r\r\nSyria, Jordan, and Iraq:...","50,000 troops\r\r\r\n214,000 reserves\r\r\r\n3...",L,1967.0


In [107]:
war_list = []
battle_list = []
def commander_war(general):
#     print general
    war = 0
    global df_battles
    df_dbg = df_run[df_run.belligerent == general]
    df_battles = pd.DataFrame()
    for battle in df_dbg.Battle.unique():
        global row
        row = df_dbg[df_dbg.Battle == battle].reset_index(drop = True)
        
        battle = row.loc[0,'Battle']

        df_str = df_lookup[df_lookup.Battle == battle].reset_index(drop = True)
        
        outcome = pd.DataFrame(row.loc[:,'VorD']).iloc[0,0]
        
        
        if row.loc[0, 'pos'] == 'L':
            opp_row = df_str[df_str.pos == 'R']
        if row.loc[0, 'pos'] == 'R':
            opp_row = df_str[df_str.pos == 'L']
            
       
            
        
        year = row.loc[0, 'year']

        own = row.drop(['Battle', 'Date', 'Location', 'Result', 'belligerent', 'opp', 'own', 'pos', 'VorD', 'year'], axis = 1)

        opp = opp_row.drop(['Battle', 'Date', 'Location', 'Result', 'belligerent', 'opp', 'own', 'pos', 'VorD', 'year'], axis = 1)
        
        opp = opp.rename(columns = {'Infantry':'infantry_opp', 
                                    'Cavalry':'cavalry_opp', 
                                    'Artillery':'artillery_opp', 
                                    'Ships':'ships_opp', 
                                    'Airforce':'airforce_opp',
                                    'Special':'special_opp'})

        opp = opp.reset_index(drop= True)

        pred_row = pd.concat([own, opp], axis = 1)
        
        pred_diff = pd.DataFrame()
        
        pred_diff['infantry_diff'] = (pred_row['Infantry'] - pred_row['infantry_opp']) / (pred_row['Infantry'] + pred_row['infantry_opp'])
        pred_diff['cavalry_dfff'] = (pred_row['Cavalry'] - pred_row['cavalry_opp']) / (pred_row['Cavalry'] + pred_row['cavalry_opp'])
        pred_diff['artillery_diff'] = (pred_row['Artillery'] - pred_row['artillery_opp']) / (pred_row['Artillery'] + pred_row['artillery_opp'])
        pred_diff['ships_diff'] = (pred_row['Ships'] - pred_row['ships_opp']) / (pred_row['Ships'] + pred_row['ships_opp'])
        pred_diff['airforce_diff'] = (pred_row['Airforce'] - pred_row['airforce_opp']) / (pred_row['Airforce'] + pred_row['airforce_opp'])
        pred_diff['special_diff'] = (pred_row['Special'] - pred_row['special_opp']) / (pred_row['Special'] + pred_row['special_opp'])

#         pred_diff = pred_diff.drop(['Infantry', 'Cavalry', 'Artillery', 'Ships', 'Airforce', 'Special', 'infantry_opp', 'cavalry_opp', 'artillery_opp', 'ships_opp', 'airforce_opp', 'special_opp'], axis = 1)

        pred_diff = pred_diff.fillna(0)

        pred_diff = pred_diff.drop(['artillery_diff', 'special_diff'], axis = 1)

        df_pred = pd.DataFrame(lr.predict_proba(pred_diff))
        
#         if row.loc[0, 'Infantry'] == 1.0:
        

        if len(df_str) == 1:
            if row.loc[0, 'VorD'] == 'V':
                value = 0.5
#                 print 'UNKNOWN'
            elif row.loc[0, 'VorD'] == 'D':
                value = -.5
#                 print 'UNKNOWN'
            elif row.loc[0, 'VorD'] == 'I':
                value = 0
#                 print 'UNKNOWN'
        else:
            if df_str.Infantry.loc[0] == 1.0 or df_str.Infantry.loc[1] == 1.0:
                if row.loc[0, 'VorD'] == 'V':
                    value = 0.5
#                     print 'UNKNOWN'
                elif row.loc[0, 'VorD'] == 'D':
                    value = -.5
#                     print 'UNKNOWN'
                elif row.loc[0, 'VorD'] == 'I':
                    value = 0
#                     print 'UNKNOWN'
            else:
                if row.loc[0, 'VorD'] == 'V':
                    value = df_pred.iloc[0,0]
                elif row.loc[0, 'VorD'] == 'D':
                    value = 0 - df_pred.iloc[0,1]
                elif row.loc[0, 'VorD'] == 'I':
                    value = .5 - df_pred.iloc[0,1]
        
        

#         print u' '.join(battle).encode('utf-8').strip(' ') +': ' + str(value).encode('utf-8')

        
        
        
        war = war + value
        battle_list.append({'General':general,
                            'Battle': battle,
                            'Value': value,
                            'Year': year,
                            'Outcome': outcome})
        df_battles = pd.DataFrame(battle_list)
        
#     print 'Total: ' + str(war)
#     print u' '.join(general).encode('utf-8').strip(' ') + ': ' + str(war)
#     if len(df_dbg) > 4:
#         war_list.append({'General':general,
#                      'WAR':war})
    war_list.append({'General': general,
                    'WAR': war,
                    'Battles': len(df_dbg)})
    print(war)
    

In [108]:
war_list = []
count = 0
for general in df_run.belligerent.unique():
    count += 1
    print(str(general) + ': ' + str(count) + '/' + str(len(df_run)))
    commander_war(general)

nan: 1/10515
0
David Ben-Gurion: 2/10515
0.971442287354
Yisrael Galili: 3/10515
0.464436526043
Yaakov Dori: 4/10515
0.464436526043
Yigael Yadin: 5/10515
0.464436526043
Mickey Marcus: 6/10515
0.464436526043
Yigal Allon: 7/10515
0.464436526043
Yitzhak Rabin: 8/10515
1.02673901816
David Shaltiel: 9/10515
0.464436526043
Moshe Dayan: 10/10515
2.0862261049
Shimon Avidan: 11/10515
0.464436526043
Moshe Carmel: 12/10515
0.464436526043
Yitzhak Sadeh: 13/10515
0.464436526043
Abdul Rahman Hassan Azzam: 14/10515
-0.467428236013
Farouk of Egypt: 15/10515
-0.467428236013
Abdullah I of Jordan: 16/10515
-0.467428236013
Muzahim al-Pachachi: 17/10515
-0.467428236013
Husni al-Za'im: 18/10515
-0.467428236013
Amin al-Husseini: 19/10515
-0.467428236013
Ahmed Ali al-Mwawi: 20/10515
-0.467428236013
Mohammed Naguib: 21/10515
-0.467428236013
John Bagot Glubb: 22/10515
-0.467428236013
Habis al-Majali: 23/10515
-0.467428236013
Hasan Salama: 24/10515
-0.467428236013
Arab Liberation Army: 25/10515
-0.467428236013
Fa

-0.5
Antigonus II Gonatas: 209/10515
0.5
Oda Nobunaga: 210/10515
4.22808805111
Tokugawa Ieyasu: 211/10515
1.32772554976
Inaba Yoshimichi: 212/10515
0.476237115334
Azai Nagamasa: 213/10515
-0.860914952791
Asakura Yoshikage: 214/10515
-1.37836685313
Timur: 215/10515
0.473471765525
Shahrukh Mirza: 216/10515
0.473471765525
Khalil Sultan: 217/10515
0.473471765525
Miran Shah: 218/10515
0.473471765525
Abu Bakr (Timurid): 219/10515
0.473471765525
Sultan Huseyn: 220/10515
0.473471765525
Pir Muhammad ibn Jahangir: 221/10515
0.473471765525
Taj al-Din Shah-i Shahan Abu'l Fath: 222/10515
0.473471765525
Bayezid I: 223/10515
-0.0210334642856
Süleyman Çelebi: 224/10515
-0.476469897374
Mehmed I: 225/10515
-0.476469897374
Stefan Lazarević: 226/10515
0.00395281538778
Gannys: 227/10515
0.5
Heraclius: 228/10515
-0.684082093199
Shahrbaraz: 229/10515
0.423360884382
Shahin Vahmanzadegan: 230/10515
0.5
Eslamu: 231/10515
-0.583476447517
Fatagar: 232/10515
-0.583476447517
Theophilos (emperor): 233/10515
-0.51486

-0.0326454782438
Mohan al-Furayji: 413/10515
-0.0326454782438
Fadhil Jalil al-Barwari: 414/10515
-0.0326454782438
Muqtada al-Sadr: 415/10515
0.529651479441
Shiek Ali al-Sauidi: 416/10515
0.0296514794413
Akram al-Kabi: 417/10515
0.0296514794413
Yusuf al-Mosawi: 418/10515
0.0296514794413
John Stewart, Earl of Buchan: 419/10515
-0.619139997597
Gilbert Motier de La Fayette: 420/10515
0.443557954739
List of Marshals of France: 421/10515
0.443557954739
Thomas of Lancaster, 1st Duke of Clarence: 422/10515
-0.446527305862
Thomas Beaufort, Duke of Exeter: 423/10515
-0.446527305862
John Beaufort, 1st Duke of Somerset: 424/10515
-0.446527305862
Zhang Fei: 425/10515
1.04957308149
Zhang He: 426/10515
-0.0277892622279
Baldwin IV of Jerusalem: 427/10515
0.0461884658263
Baldwin of Ibelin: 428/10515
0
Balian of Ibelin: 429/10515
-0.972024279231
Hugh II of Saint Omer: 430/10515
0
Gökböri: 431/10515
0.88513844382
Charles I of Naples: 432/10515
0.5
Manfred, King of Sicily: 433/10515
-0.5
Quintus Fulvius F

0.527212542053
Hanno, son of Bomilcar: 604/10515
0.527212542053
Hasdrubal (quartermaster): 605/10515
0.527212542053
Marcus Claudius Marcellus: 606/10515
0.487113503304
Lucius Manlius Vulso Longus: 607/10515
0.502711766226
Hanno the Great: 608/10515
-1.0057172163
Appius Claudius Pulcher (consul 212 BC): 609/10515
-0.53121405258
Francesco I Sforza: 610/10515
1.43720306497
Micheletto Attendolo: 611/10515
0.0306059842338
William Kirkcaldy of Grange: 612/10515
1.01748092864
James Hepburn, 4th Earl of Bothwell: 613/10515
-0.50150283181
Necho II: 614/10515
-0.540286520533
Nebuchadnezzar II: 615/10515
2.03729894968
Marcus Licinius Crassus: 616/10515
-0.490927297726
Publius Licinius Crassus (son of triumvir): 617/10515
-0.490927297726
Gaius Cassius Longinus: 618/10515
-1.47244812424
Surena: 619/10515
0.487922978584
Hasan ibn al-Nu'man: 620/10515
0.5
Tiberios III: 621/10515
-0.5
Hasdrubal the Boetharch: 622/10515
-0.5
Scipio Aemilianus: 623/10515
1.0
Manius Manilius: 624/10515
0.5
Alberico da Ba

0.0505301729707
Lamba Doria: 801/10515
0.520197892385
Andrea Dandolo (admiral): 802/10515
-0.523197895705
Titus Quinctius Flamininus: 803/10515
0.983310626372
Ubba: 804/10515
-0.5
Odda, Ealdorman of Devon: 805/10515
0.5
Alcibiades: 806/10515
0.493318137701
Theramenes: 807/10515
0.493318137701
Hermocrates: 808/10515
0.00367652117782
Gar Trinring Tsendro: 809/10515
0.419498177575
Xue Rengui: 810/10515
-0.422428737716
Ashina Daozi: 811/10515
-0.422428737716
Guo Daifeng: 812/10515
-0.422428737716
Abdul Qayyum Zakir: 813/10515
-0.5
Sancho d'Avila: 814/10515
1.02655731455
Mas'ud I of Ghazni: 815/10515
-0.554082451117
Ali Daya: 816/10515
-0.554082451117
Ahmad Shirazi: 817/10515
-0.554082451117
Abu Sahl Zawzani: 818/10515
-0.554082451117
Abd al-Razzaq Maymandi: 819/10515
-0.554082451117
Begtoghdi: 820/10515
-0.554082451117
Subashi (general): 821/10515
-0.554082451117
Chaghri Beg: 822/10515
0.55111004685
Tughril: 823/10515
0.55111004685
Faramurz: 824/10515
0.55111004685
Minamoto no Yoshitsune: 

...

Minister of Defence (India): 5692/10515
0.467994469185
Sam Manekshaw: 5693/10515
0.467994469185
Chief of the Army Staff (India): 5694/10515
0.420269905576
Jagjit Singh Aurora: 5695/10515
0.467994469185
General officer commanding: 5696/10515
0.420269905576
Eastern Command (India): 5697/10515
0.467994469185
Gopal Gurunath Bewoor: 5698/10515
0.467994469185
Southern Command (India): 5699/10515
0.467994469185
Kunhiraman Palat Candeth: 5700/10515
0.467994469185
Western Command (India): 5701/10515
0.420269905576
Manohar Lal Chibber: 5702/10515
0.467994469185
Northern Command (India): 5703/10515
0.467994469185
Premindra Singh Bhagat: 5704/10515
0.467994469185
Central Command (India): 5705/10515
0.467994469185
Sagat Singh: 5706/10515
0.467994469185
IV Corps (India): 5707/10515
0.467994469185
Tapishwar Narain Raina: 5708/10515
0.467994469185
II Corps (India): 5709/10515
0.467994469185
Sartaj Singh (general): 5710/10515
0.467994469185
XV Corps (India): 5711/10515
0.420269905576
I Corps (India): 5

-1.0
Victor Banjo: 5865/10515
-0.5
Albert Okonkwo: 5866/10515
-0.5
Festus Akagha: 5867/10515
-1.0
Humphrey Chukwuka: 5868/10515
-0.5
Benjamin Adekunle: 5869/10515
0.457607728661
Ma Zhanshan: 5870/10515
-0.556442045261
Feng Zhanhai: 5871/10515
-0.556442045261
Shigeru Honjō: 5872/10515
0.553472694138
Jirō Minami: 5873/10515
0.553472694138
Juan José Latorre: 5874/10515
0.473497355392
Galvarino Riveros: 5875/10515
0.427365963938
Shizuo Sakaguchi: 5876/10515
0.357690492125
Cornelis van den Hoogenband: 5877/10515
-0.360457381975
William A. Glassford: 5878/10515
-0.360457381975
Paul H. Talbot: 5879/10515
-0.360457381975
Pavlos Kountouriotis: 5880/10515
1.0
Daniel J. Callaghan: 5881/10515
0.544525520843
Willis Augustus Lee: 5882/10515
0.544525520843
Frank R. Walker: 5883/10515
-0.408382498363
Aurelio García: 5884/10515
-0.049109790808
Sackville Carden: 5885/10515
0.5
Émile Paul Amable Guépratte: 5886/10515
0.5
Siad Barre: 5887/10515
-0.503643317711
Mohammad Ali Samatar: 5888/10515
-0.503643317

Fedlim Ó Conchobair: 6061/10515
-0.5
Tadhg Ó Cellaigh: 6062/10515
-0.5
List of monarchs of Thomond: 6063/10515
-0.5
Ualgarg Mór Ó Ruairc: 6064/10515
-0.5
Richard F. Natonski: 6065/10515
0.442155744715
Keith J. Stalder: 6066/10515
0.442155744715
Abu Musab al-Zarqawi: 6067/10515
-0.445123217936
Gnaeus Fulvius Centumalus Maximus: 6068/10515
-0.490117608708
Leo II, King of Armenia: 6069/10515
-0.5
Demetrius II of Georgia: 6070/10515
-0.5
Al-Mansur Qalawun: 6071/10515
0.5
Mohamed Afrah Qanyare: 6072/10515
-0.5
Musa Sudi Yalahow: 6073/10515
-0.5
Nuur Daqle: 6074/10515
-0.5
Rao Qamar Suleman: 6075/10515
0.412688541061
Masood Aslam: 6076/10515
0.412688541061
Hifazat Ullah Khan: 6077/10515
0.412688541061
Muhammad Haroon Aslam: 6078/10515
0.412688541061
Sajjad Ghani: 6079/10515
0.412688541061
Ijaz Awan: 6080/10515
0.412688541061
Fazlullah (militant leader): 6081/10515
-0.415605588206
Abu Saeed (TNSM Commander): 6082/10515
-0.415605588206
Misbah ud-Din: 6083/10515
-0.415605588206
Sultan Khan (Com

-0.5
Yapancha beg: 6258/10515
-0.5
Zaynash morza: 6259/10515
-0.5
Qolsharif: 6260/10515
-0.5
Afonso Pereira de Lacerda: 6261/10515
0.5
Diogo de Melo Coutinho: 6262/10515
0.5
Gabriel, comte de Montgomery: 6263/10515
0
Pieter Adriaanszoon van der Werff: 6264/10515
0.514259239608
Afonso I of Portugal: 6265/10515
0.5
Hervey de Glanvill: 6266/10515
0.5
Arnout IV, Count of Aarschot: 6267/10515
0.5
Christian of Ghistelles: 6268/10515
0.5
Simon of Dover: 6269/10515
0.5
Saher of Archelle: 6270/10515
0.5
Louis XIV of France: 6271/10515
1.22903141597
Jacques de Fariaux, baron de Maulde: 6272/10515
-0.827980706693
Alaal-Din: 6273/10515
-0.5
Leonis Pereira: 6274/10515
0.5
Jean Parisot de Valette: 6275/10515
0.563959558037
Jean de la Cassière: 6276/10515
0.563959558037
Mathurin Romegas: 6277/10515
0.563959558037
Goncales de Medran: 6278/10515
0.563959558037
Melchior de Robles: 6279/10515
0.563959558037
García de Toledo Osorio, 4th Marquis of Villafranca: 6280/10515
0.563959558037
Decimus Junius Brut

0.5
Diwan Mulraj Chopra: 6447/10515
0.5
Ferdinand II, Holy Roman Emperor: 6448/10515
-0.56174167428
Tiryaki Hasan Pasha: 6449/10515
0.558779669498
Octavius Ignatius, Prince of Arenberg and Barbançon: 6450/10515
-0.409782711559
Menno van Coehoorn: 6451/10515
0.0430666433002
Alexander II of Russia: 6452/10515
0.465778396389
Carol I of Romania: 6453/10515
0.465778396389
Grand Duke Nicholas Nikolaevich of Russia (1831–1891): 6454/10515
0.465778396389
Eduard Totleben: 6455/10515
-0.000303239890517
Nogi Maresuke: 6456/10515
0.458224411312
Franklin Gardner: 6457/10515
-0.435553622825
Joaquín Rea: 6458/10515
-0.508333973287
Thomas Childs: 6459/10515
0.505328870749
Calixto García: 6460/10515
0.483862965326
José Toral y Velázquez: 6461/10515
-0.486866053699
Alija Izetbegović: 6462/10515
-0.0713886141177
Sefer Halilović: 6463/10515
-0.0713886141177
Rasim Delić: 6464/10515
-0.0713886141177
Jovan Divjak: 6465/10515
-0.0713886141177
Enver Hadžihasanović: 6466/10515
-0.0713886141177
Mustafa Hajrulaho

In [109]:
pred_row

NameError: name 'pred_row' is not defined

In [ ]:
row.loc[0, 'Battle']

In [ ]:
df_run[df_run.belligerent == 'Alexane']

In [110]:
df_totalwar = pd.DataFrame(war_list).sort_values('WAR', ascending = False).reset_index(drop = True)

In [154]:
df_totalwar[df_totalwar.General == 'Alexander the Great']

,Battles,General,WAR
9,9,Alexander the Great,4.374161


In [ ]:
df_totalwar[df_totalwar.General == 'Ariel Sharon']

In [ ]:
df_totalwar

In [ ]:
std = np.std(df_totalwar.WAR)

In [ ]:
tw_mean = df_totalwar.WAR.mean()

In [ ]:
(16.678901 - tw_mean) / std

In [ ]:
(7.428568 - tw_mean) / std

In [ ]:
def graph(commander):
    battle_list = []
    global df_battles
    df_battles = pd.DataFrame()
    commander_war(commander)

    df_battles.Year = df_battles.Year.fillna(-5000)
    df_battles.Year = df_battles.Year.astype(float, errors = 'ignore')
    df_battles = df_battles.sort_values('Year')
    wl = pd.DataFrame(df_battles.Outcome.value_counts())
    
    if 'V' in wl.index:
        wins = wl.loc['V', 'Outcome']
    else:
        wins = 0
    if 'D' in wl.index:
        losses = wl.loc['D', 'Outcome']
    else:
        losses = 0
    if 'I' in wl.index:
        draws = wl.loc['I', 'Outcome']
    else:
        draws = 0
    df_battles = df_battles.reset_index(drop = True)
    for i, row in df_battles.iterrows():
        df_battles.ix[i, 'cumulative'] = sum(df_battles.Value[0:i+1])

    plt.figure(figsize = (12,9))
    plt.style.use('fivethirtyeight')
    plt.bar(df_battles.index, 
            df_battles['Value'], 
            label = 'Battle Value', 
            color = df_battles.Outcome.map({'V':'blue', 'I':'orange', 'D':'red'}), 
            alpha = .5, 
            zorder = 2)
    if max(df_battles.cumulative) < 1 and min(df_battles.cumulative) > 1:
        plt.ylim(-1,1)
    plt.plot(df_battles.index, df_battles['cumulative'], color = 'black', label = 'Cumulative WARG', zorder = 1)
#     plt.scatter(df_battles.index, df_battles['cumulative'], s = 100, zorder = 2)
    plt.xticks(df_battles.index, df_battles['Battle'], rotation = 'vertical')

    plt.title(commander + ' ({} - {} - {})'.format(wins, draws, losses))
    plt.tight_layout()
    
    plt.savefig('{}_warg.png'.format(commander))

In [ ]:
df_run = df_run.dropna(subset = ['belligerent'])

In [ ]:
battle_list = []
graph('Moshe Dayan')

In [ ]:
df_battles

In [ ]:
df_model[df_model.Battle == 'Battle_of_Borodino'].transpose()

In [ ]:
df_model[df_model.Battle == 'Six Day War']

In [ ]:
df_battles

In [ ]:
df_war = pd.DataFrame(war_list).sort_values('WAR', ascending = False).reset_index(drop = True)

In [ ]:
df_war.head()

In [151]:
df_totalwar = df_totalwar[df_totalwar.General != 'Capital punishment']
df_totalwar = df_totalwar[df_totalwar.General != 'Lieutenant general']

In [ ]:
df_totalwar[df_totalwar.General == 'Erwin Rommel']

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(df_totalwar['WAR'], kde = False, bins = 30)
plt.title("Distribution of Generals' War")
plt.ylabel('Number of Generals')
plt.annotate('Napoleon', xy = (16.713, 1), xytext = (17.713, 400), arrowprops = dict(facecolor = 'black'))
plt.annotate('Julius Caesar', xy = (7.364545, 1), xytext = (8.364545, 400), arrowprops = dict(facecolor = 'black'))
plt.tight_layout()
plt.savefig('/home/ethanarsht/Downloads/distplot.png')

In [152]:
output_file('docs/index.html')
source = ColumnDataSource(data = dict(battles = df_totalwar.Battles,
                                      war = df_totalwar.WAR,
                                      general = df_totalwar.General))

hover = HoverTool(tooltips = [('General', '@general'),
                              ('Battles', '@battles'),
                              ('WAR', '@war'),
                                  ])
p = figure(title = 'WAR vs Number of Battles',
           plot_height = 600, 
           plot_width = 800,
          
           tools = [hover, "tap", 'box_zoom'])
p.circle(x = 'war', y = 'battles', size = 10, source = source)
p.xaxis.axis_label = 'WAR'
p.yaxis.axis_label = 'Battles'

link = "https://ethanarsht.github.io/military_rankings/{}.html".format('@general')

if link == 'https://ethanarsht.github.io/military_rankings/Thomas de Courtenay, 5th/13th Earl of Devon':
        link = 'https://ethanarsht.github.io/military_rankings/Thomas de Courtenay'
url = link
taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)
save(p)

'/home/ethanarsht/military_rankings/docs/index.html'

In [153]:
df_battles

,Battle,General,Outcome,Value,Year,cumulative
0,Battle of Arras,Harold Franklyn,D,-0.430169,1940.0,-0.430169


In [189]:
def bokeh(commander):
    global df_battles
    
    df_battles = pd.DataFrame()
    commander_war(commander)
    
#     df_battles = df_battles[df_battles.General == commander]
    df_battles.Year = df_battles.Year.fillna(-5000)
    df_battles.Year = df_battles.Year.astype(float, errors = 'ignore')
    df_battles = df_battles.sort_values('Year')
    wl = pd.DataFrame(df_battles.Outcome.value_counts())
    
    if 'V' in wl.index:
        wins = wl.loc['V', 'Outcome']
    else:
        wins = 0
    if 'D' in wl.index:
        losses = wl.loc['D', 'Outcome']
    else:
        losses = 0
    if 'I' in wl.index:
        draws = wl.loc['I', 'Outcome']
    else:
        draws = 0
    df_battles = df_battles.reset_index(drop = True)
    for i, row in df_battles.iterrows():
        df_battles.ix[i, 'cumulative'] = sum(df_battles.Value[0:i+1])
    
    
    source = ColumnDataSource(data = dict(battle = df_battles.Battle,
                              general = df_battles.General,
                              outcome = df_battles.Outcome,
                              value = df_battles.Value,
                              year = df_battles.Year,
                              cumulative = df_battles.cumulative))
    if all(x != -5000 for x in df_battles.Year):
        hover = HoverTool(tooltips = [('Battle', '@battle'),
                                  ('Outcome', '@outcome'),
                                  ('WAR', '@value'),
                                  ('Cumulative WAR', '@cumulative'),
                                  ('Year', '@year')])
    else:
        hover = HoverTool(tooltips = [('Battle', '@battle'),
                                  ('Outcome', '@outcome'),
                                  ('WAR', '@value'),
                                  ('Cumulative WAR', '@cumulative')
                                  ])
    
    x = list(df_battles.Battle)
    y = list(df_battles.Value)
    y_cum = list(df_battles.cumulative)
    
    if commander == 'Thomas de Courtenay, 5th/13th Earl of Devon':
        commander = 'Thomas de Courtenay'
    
    output_file('docs/{}.html'.format(commander))
#     output_file('{}.html'.format(commander))
    
    p = figure(title = commander + ' ({} - {} - {})'.format(wins, draws, losses), 
               plot_width=800, 
               plot_height=600, 
               x_range = x, 
               tools = [hover, "tap"])
#     p.xaxis.axis_label = "General's Biography: en.wikipedia.org/wiki/{}".format(commander)
    
    p.line(x = 'battle', y = 'cumulative', source = source, line_width = 5)
#     url = 'en.wikipedia.org/wiki/{}'.format(commander)
#     taptool = p.select(type=TapTool)
#     taptool.callback = OpenURL(url=url)
    
    color_mapper = CategoricalColorMapper(factors=['V', 'D', 'I'], palette=['Blue', 'Red', 'Yellow'])
    p.vbar(x = 'battle', 
           top = 'value', 
           source = source, 
           width = 0.9, 
           color={'field': 'outcome', 'transform': color_mapper},
           alpha = 0.5)
    url = 'en.wikipedia.org/wiki/@battle'
    taptool = p.select(type=TapTool)
    taptool.callback = OpenURL(url=url)
    
    p.xaxis.major_label_orientation = math.pi/2
    

#     show(p)
    save(p)

In [186]:
battle_list = []
bokeh('Napoleon')

16.678282131


/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [160]:
df_battles

,Battle,General,Outcome,Value,Year,cumulative
0,Siege of Toulon,Napoleon,V,0.479531,1793.0,0.479531
1,Battle of Montenotte,Napoleon,V,0.478016,1796.0,0.957547
2,Battle of Castiglione,Napoleon,V,0.489183,1796.0,1.446729
3,Battle of Lodi,Napoleon,V,0.473926,1796.0,1.920655
4,Battle of Bassano,Napoleon,V,0.468786,1796.0,2.389441
5,Battle of the Bridge of Arcole,Napoleon,V,0.507813,1796.0,2.897254
6,Battle of Rivoli,Napoleon,V,0.518113,1797.0,3.415367
7,Battle of the Pyramids,Napoleon,V,0.518009,1798.0,3.933376
8,Siege of El Arish,Napoleon,V,0.572390,1799.0,4.505766
9,Siege of Acre (1799),Napoleon,V,0.578863,1799.0,5.084629


In [ ]:

count = 0
for general in df_run.belligerent.unique()[1:]:
    battle_list = []
    count += 1
    print(str(count) + '/' + str(len(df_run.belligerent.unique())))
    print(general)
    bokeh(general)

1/6620
David Ben-Gurion
0.971442287354
2/6620
Yisrael Galili
0.464436526043


/home/ethanarsht/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


3/6620
Yaakov Dori
0.464436526043
4/6620
Yigael Yadin
0.464436526043
5/6620
Mickey Marcus
0.464436526043
6/6620
Yigal Allon
0.464436526043
7/6620
Yitzhak Rabin
1.02673901816
8/6620
David Shaltiel
0.464436526043
9/6620
Moshe Dayan
2.0862261049
10/6620
Shimon Avidan
0.464436526043
11/6620
Moshe Carmel
0.464436526043
12/6620
Yitzhak Sadeh
0.464436526043
13/6620
Abdul Rahman Hassan Azzam
-0.467428236013
14/6620
Farouk of Egypt
-0.467428236013
15/6620
Abdullah I of Jordan
-0.467428236013
16/6620
Muzahim al-Pachachi
-0.467428236013
17/6620
Husni al-Za'im
-0.467428236013
18/6620
Amin al-Husseini
-0.467428236013
19/6620
Ahmed Ali al-Mwawi
-0.467428236013
20/6620
Mohammed Naguib
-0.467428236013
21/6620
John Bagot Glubb
-0.467428236013
22/6620
Habis al-Majali
-0.467428236013
23/6620
Hasan Salama
-0.467428236013
24/6620
Arab Liberation Army
-0.467428236013
25/6620
Fawzi al-Qawuqji
-0.467428236013
26/6620
Menachem Begin
-0.0689526608217
27/6620
Ariel Sharon
2.14131954943
28/6620
Rafael Eitan
-0.06

-0.860914952791
213/6620
Asakura Yoshikage
-1.37836685313
214/6620
Timur
0.473471765525
215/6620
Shahrukh Mirza
0.473471765525
216/6620
Khalil Sultan
0.473471765525
217/6620
Miran Shah
0.473471765525
218/6620
Abu Bakr (Timurid)
0.473471765525
219/6620
Sultan Huseyn
0.473471765525
220/6620
Pir Muhammad ibn Jahangir
0.473471765525
221/6620
Taj al-Din Shah-i Shahan Abu'l Fath
0.473471765525
222/6620
Bayezid I
-0.0210334642856
223/6620
Süleyman Çelebi
-0.476469897374
224/6620
Mehmed I
-0.476469897374
225/6620
Stefan Lazarević
0.00395281538778
226/6620
Gannys
0.5
227/6620
Heraclius
-0.684082093199
228/6620
Shahrbaraz
0.423360884382
229/6620
Shahin Vahmanzadegan
0.5
230/6620
Eslamu
-0.583476447517
231/6620
Fatagar
-0.583476447517
232/6620
Theophilos (emperor)
-0.514865232425
233/6620
Theophobos
-0.514865232425
234/6620
Manuel the Armenian
-0.514865232425
235/6620
Khaydhar ibn Kawus al-Afshin
0.511861715859
236/6620
Umar al-Aqta
0.511861715859
237/6620
Jaysh ibn Samsama
0.5
238/6620
Damian Da

-0.619139997597
419/6620
Gilbert Motier de La Fayette
0.443557954739
420/6620
List of Marshals of France
0.443557954739
421/6620
Thomas of Lancaster, 1st Duke of Clarence
-0.446527305862
422/6620
Thomas Beaufort, Duke of Exeter
-0.446527305862
423/6620
John Beaufort, 1st Duke of Somerset
-0.446527305862
424/6620
Zhang Fei
1.04957308149
425/6620
Zhang He
-0.0277892622279
426/6620
Baldwin IV of Jerusalem
0.0461884658263
427/6620
Baldwin of Ibelin
0
428/6620
Balian of Ibelin
-0.972024279231
429/6620
Hugh II of Saint Omer
0
430/6620
Gökböri
0.88513844382
431/6620
Charles I of Naples
0.5
432/6620
Manfred, King of Sicily
-0.5
433/6620
Quintus Fulvius Flaccus (consul 237 BC)
-0.0510999945234
434/6620
Hanno the Elder
-0.98518466264
435/6620
Tiberius Sempronius Gracchus (consul 215 BC)
0.499063345052
436/6620
Manius Curius Dentatus
0.5
437/6620
John II Komnenos
0.5
438/6620
Telerig of Bulgaria
-0.426298612591
439/6620
Seron (Syrian governor)
-0.562658880594
440/6620
Judas Maccabeus
2.3033150503

...

2198/6620
Vladimir II Monomakh
-0.5
2199/6620
Rostislav Vsevolodovich
-0.5
2200/6620
Vishvamitra
-0.5
2201/6620
Sudas
0.5
2202/6620
Vashistha
0.5
2203/6620
Arminius
-0.00638242454983
2204/6620
Publius Quinctilius Varus
-0.496622952787
2205/6620
Suicide
-0.496622952787
2206/6620
Jean de Beaumanoir
0.49849716819
2207/6620
Robert Bemborough
-0.50150283181
2208/6620
Tiberius Sempronius Longus (consul 218 BC)
-0.503751881471
2209/6620
Abu Sufyan ibn Harb
-0.643217168625
2210/6620
Uthman
0.553472694138
2211/6620
Salman the Persian
0.553472694138
2212/6620
Arnegisclus
-0.5
2213/6620
Attila
0.5
2214/6620
Edigu
0.456971910728
2215/6620
Temür Qutlugh
0.456971910728
2216/6620
Tokhtamysh
-0.459956832578
2217/6620
Germanicus
0.5
2218/6620
Flavus (son of Segimerus)
0.5
2219/6620
Chariovalda
0.5
2220/6620
Inguiomer
-0.5
2221/6620
Richomeres
0
2222/6620
Profuturus
0
2223/6620
Traianus (magister peditum)
0
2224/6620
Sueridas
0
2225/6620
Colias
0
2226/6620
Hugh O'Neill, Earl of Tyrone
0.487113503304
222

2396/6620
Wei Yan
0.032610350953
2397/6620
Yang Yi
0.032610350953
2398/6620
Huang Zu
-0.0741509596224
2399/6620
Ling Cao
0.0712088046161
2400/6620
Aju
0.412122538595
2401/6620
Arikhgiya
0.412122538595
2402/6620
Shi Tianze
0.412122538595
2403/6620
Lü Wenhuan
-0.415038412226
2404/6620
Li Tingzhi
-0.415038412226
2405/6620
Shamoke
-0.490117608708
2406/6620
Cao Shuang
-0.5
2407/6620
Xu Rong (general)
0.49849716819
2408/6620
Oda Nobutaka
0.45808098525
2409/6620
Nakamura Kazuuji
0.45808098525
2410/6620
Horio Yoshiharu
0.45808098525
2411/6620
Katō Mitsuyasu
0.45808098525
2412/6620
Kimura Shigeori
0.45808098525
2413/6620
Takayama Shigetama
0.45808098525
2414/6620
Nakagawa Kiyohide
0.45808098525
2415/6620
Mikoda Masaharu
0.45808098525
2416/6620
Toyotomi Hidenaga
0.45808098525
2417/6620
Kuroda Yoshitaka
1.40990012978
2418/6620
Akechi Shigetomo
-0.461067000526
2419/6620
Abe Sadamasa
-0.461067000526
2420/6620
Shibata Katsusada
-0.461067000526
2421/6620
Saitō Toshimitsu
-0.461067000526
2422/6620
Tsu

2592/6620
Hugh Judson Kilpatrick
0.469691353053
2593/6620
Thomas T. Munford
0.928802529824
2594/6620
George Gordon, 2nd Marquess of Huntly
0.506722952009
2595/6620
Manus O'Cahan's Regiment
1.05541106795
2596/6620
William Baillie (soldier)
-1.09701995498
2597/6620
Alexander Lindsay, 1st Earl of Balcarres
-0.509727802207
2598/6620
Sardar
-0.0789786108868
2599/6620
Ranjodh Singh Majithia
-1.05145157197
2600/6620
Sir Harry Smith, 1st Baronet
0.981048080429
2601/6620
Prince Frederick, Duke of York and Albany
-0.525523370112
2602/6620
Ivan Essen
0.47486944194
2603/6620
Guillaume Brune
0.514859637516
2604/6620
Herman Willem Daendels
-0.461694872013
2605/6620
Samuel Gibbs French
-0.527167354681
2606/6620
John M. Corse
0.522665880026
2607/6620
Jacques Marcus Prevost
-0.071757687065
2608/6620
Thomas Brown (loyalist)
0.419498177575
2609/6620
James Screven
-0.422428737716
2610/6620
Jacques Leroy de Saint Arnaud
0.482213854291
2611/6620
FitzRoy Somerset, 1st Baron Raglan
1.50516020189
2612/6620
Ale

2769/6620
Jacques MacDonald
-1.53192928881
2770/6620
Khem Karan Singh
0.966491637375
2771/6620
Arun Shridhar Vaidya
0.49849716819
2772/6620
Abdul Ali Malik
-0.972491774357
2773/6620
Dagobert Sigmund von Wurmser
-0.963968226994
2774/6620
Jonathan M. Wainwright (general)
-1.45686754049
2775/6620
George M. Parker (general)
-0.52513055806
2776/6620
Edward P. King
-0.52513055806
2777/6620
Vicente Lim
-1.02709814541
2778/6620
Masaharu Homma
1.44791907813
2779/6620
Susumu Morioka
0.522131608288
2780/6620
Kineo Kitajima
0.522131608288
2781/6620
Kameichiro Nagano
0.522131608288
2782/6620
Nikolay Kamensky
0.994041852967
2783/6620
Frederick Dobson Middleton
-0.00880613106755
2784/6620
Bowen van Straubenzee
0.440249933983
2785/6620
Gabriel Dumont (Métis leader)
0.459840582987
2786/6620
Louis Riel
-0.443214778906
2787/6620
Bernardo de Gálvez
1.83079653737
2788/6620
Alexander Dickson (American War of Independence)
-0.482532423115
2789/6620
John C. Breckinridge
0.969498177103
2790/6620
Charles Read (

-1.01661826886
2951/6620
John Forbes (Portuguese general)
-1.01661826886
2952/6620
Winfield Scott Hancock
-0.547876476355
2953/6620
Henry Heth
0.00628733938035
2954/6620
William Howe, 5th Viscount Howe
2.78820896037
2955/6620
Wilhelm von Knyphausen
0.441518400262
2956/6620
George Washington
-0.102971851557
2957/6620
Nathanael Greene
-1.02422153819
2958/6620
John Sullivan (general)
-1.02418235201
2959/6620
William Alexander, Lord Stirling
-1.47757688203
2960/6620
Adam Stephen
-0.49843869624
2961/6620
Anthony Wayne
0.464456442642
2962/6620
Casimir Pulaski
-1.11800327044
2963/6620
Moses Hazen
-0.54798122821
2964/6620
Radomir Putnik
1.95039202412
2965/6620
Petar Bojović
0.927966172833
2966/6620
Alexander I of Yugoslavia
0.941976131747
2967/6620
Živojin Mišić
0.982793365713
2968/6620
Janko Vukotić
0.478268170728
2969/6620
Mihail Savov
-0.981268913289
2970/6620
Stiliyan Kovachev
-0.481268913289
2971/6620
Vicho Dikov
-0.981268913289
2972/6620
Gustavus Adolphus of Sweden
0.952774308992
2973/66

In [ ]:
mka = df_run.loc[8965, 'belligerent']

In [ ]:
mka

In [ ]:
df_battles

In [ ]:
battle_list = []
bokeh(u'Ariel Sharon')

In [ ]:
df_warchart = pd.DataFrame({'Labels': ['Win = 1', 'Replacement-Level Performance = .51', 'WAR From Battle = .49'],
              'Values': [1, .51, .49]})

In [ ]:
df_warchart = df_warchart.sort_values('Values', ascending = True)

In [ ]:
df_warchart

In [ ]:
df_warchart = df_warchart.iloc[::-1]

In [ ]:
fig = plt.figure(figsize = (12,9))
plt.style.use('fivethirtyeight')
plt.bar(df_warchart.index, df_warchart.Values, width = 1, color = ['red', 'blue', 'yellow'])
plt.xticks(df_warchart.index + .5, df_warchart.Labels, rotation = 45)
plt.xticks()
plt.title("Bonaparte's WAR in Battle of Borodino")
plt.tight_layout()
plt.savefig('napoleon_war_sample')

# ax.set_axis_bgcolor('gray')

In [ ]:
df_warchart_two = pd.DataFrame({'Labels': ['Win = -1', 'Replacement-Level Performance = .49', 'WAR From Battle = -.51'],
              'Values': [-1, -.49, -.51]})

In [ ]:
fig = plt.figure(figsize = (12,9))
plt.style.use('fivethirtyeight')
plt.bar(df_warchart_two.index, df_warchart_two.Values, width = 1, color = ['red', 'blue', 'white'])
plt.xticks(df_warchart_two.index, df_warchart_two.Labels, rotation = 45)
plt.title("Kutuzov's WAR in Battle of Borodino")
plt.tight_layout()
plt.savefig('/home/ethanarsht/Downloads/kutuzov_war_sample')

# ax.set_axis_bgcolor('gray')